In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from pandas.tseries.offsets import DateOffset
%matplotlib inline
import os

from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.chdir("F:\\Data analysis\\Imarticus data\\Capstone project")
data_og=pd.read_excel("Sales_Forecast_Data.xlsx",parse_dates = True,squeeze= True)
rainfall_og=pd.read_excel("StateWise Rainfall.xlsx")
gdp_og=pd.read_excel("StateWise GDP.xlsx")

In [ ]:
data=data_og.copy()
rainfall=rainfall_og.copy()
gdp=gdp_og.copy()

In [ ]:
data.isna().sum()

## Create new column with date format

In [ ]:
first_mon=['Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
data_abc1 = data[data["MONTH"].isin(first_mon)]

data_abc1['Year-Month']= data_abc1['MONTH'] + data_abc1['FIN_YEAR'].str[:4]

sec_mon=['Jan','Feb','Mar']
data_abc2 = data[data["MONTH"].isin(sec_mon)]

data_abc2['Year-Month']= data_abc2['MONTH'] + data_abc2['FIN_YEAR'].str[5:]

data_abc = pd.concat([data_abc1, data_abc2])

data_abc['date']= pd.to_datetime(data_abc['Year-Month'],infer_datetime_format=True)

data_abc.head()

Importing rainfall and gdp data in sales data

## Removing blank spaces from COMPANY Column

In [ ]:
data_abc['COMPANY'] = data_abc['COMPANY'].str.replace(' ','')

In [ ]:
df = data_abc.drop(['FIN_YEAR','MONTH','Year-Month','DISTRICT'],axis=1)
df.set_index('date',inplace=True)

# Ploting graph for companies having max sales (TOP 5)

In [ ]:
df.head()

In [ ]:
company_sales_order = df.groupby(['COMPANY'])['VALUE'].sum()

In [ ]:
company_sales_order.sort_values(ascending=False).head()

In [ ]:
top_com = company_sales_order.sort_values(ascending=False).head()

In [ ]:
for a in top_com.index:
    df_comp = df[df['COMPANY']==a]
    df_comp = df_comp.groupby('date')['VALUE'].sum()
    df_comp.plot(legend=True,label=a,figsize=(18,8))

# ABC Manufacturing Sales across given states

In [ ]:
df_abc = df[df['COMPANY']=='ABCManufacturing']

In [ ]:
state = list(df_abc['STATE'].unique())

In [ ]:
for i in state:
    df_hr = df_abc[df_abc['STATE']==i]
    df_hr = df_hr.groupby('date')['VALUE'].sum()
    df_hr.plot(legend=True,label=i,figsize=(18,8))

## Rainfall GDP Crrelation with sales of fertilizers

In [ ]:
gdp['date']=pd.to_datetime(gdp['Year-Month'],infer_datetime_format=True)

In [ ]:
rainfall['date']=pd.to_datetime(rainfall['Year-Month'],infer_datetime_format=True)

In [ ]:
rainfall.head()

In [ ]:
gdp.head()

In [ ]:
rain_gdp = pd.merge(gdp,rainfall,how='inner',on=['date','STATE','Year-Month'])

In [ ]:
rain_gdp.head()

In [ ]:
data_abc.head()

In [ ]:
data_abc_corr= pd.merge(data_abc,rain_gdp,how='inner',on=['date','STATE','Year-Month'])

In [ ]:
data_abc_corr.drop(['FIN_YEAR','MONTH','FG','DISTRICT','Year-Month'],axis=1,inplace=True)

In [ ]:
data_abc_corr.head()

In [ ]:
data_abc_corr.corr()

# ABC Manufacturing company time series decomposition

In [ ]:
data_abc_sales = data_abc.groupby(['date'])['VALUE'].sum()

In [ ]:
data_abc_decompose = sm.tsa.seasonal_decompose(data_abc_sales,model='additive', period=12)

In [ ]:
fig = data_abc_decompose.plot()
fig.set_size_inches((16, 8))
# Tight layout to realign things
fig.tight_layout()
plt.show()

# Creating adfuller test function for checking the series stationarity

In [ ]:
def adfuller_test(state_name):
    df_1 = df_abc[df_abc['STATE']==state_name]
    df_1 = df_1.groupby(['date'])['VALUE'].sum()
    if adfuller(df_1)[1] <0.05:
        print("The p-value for ",state_name," time series is",adfuller(df_1)[1],"(<0.05).Hence the series is stationary")
    else:
        print("The p-value for ",state_name," time series is",adfuller(df_1)[1],"(<0.05).Hence the series is NOT stationary")

In [ ]:
for i in state:
    adfuller_test(i)

# Creating function for predicting 

In [ ]:
def SARIMA_MODEL(state_name, prediction_num):
    data = df_abc[df_abc['STATE']==state_name]
    data = data.groupby(['date'])['VALUE'].sum()
    data = data.to_frame()
    data.columns=['Sales']
 
    if adfuller(data)[1] <0.05:
        print("The p-value for ",state_name," time series is",adfuller(data)[1],"(<0.05).Hence the series is stationary")
        data_stat = data.copy()
    else:
        print("The p-value for ",state_name," time series is",adfuller(data)[1],"(<0.05).Hence the series is NOT stationary")
        for i in range(0,12):
            data['First Difference']=data['Sales']-data['Sales'].shift(i)
        
            if adfuller(data['First Difference'].dropna())[1] < 0.05:
                print("Using differencing method with differenceing order ",i," our data got stationary with p-value of",adfuller(data['First Difference'].dropna())[1])
            
                data_stat = data['First Difference'].dropna()
                data_stat.columns = ['Sales']
            
                data.drop(['First Difference'],axis=1,inplace=True)
                data.columns = ['Sales']
                break
    
    train = data_stat[:round(0.75*(len(data_stat)))]
    test = data_stat[round(0.75*(len(data_stat))):]
    
    import itertools
    p=range(0,8) #for AR
    q=range(0,8) #for MA
    d=range(0,2) 

    pdq_combination=list(itertools.product(p,d,q))

    rmse=[]
    mape=[]
    order1=[]
    
    for pdq in pdq_combination:
        try:
            model=sm.tsa.statespace.SARIMAX(train,order=pdq).fit()
            pred=model.predict(start=len(train),end=(len(data_stat)-1))
            error_rmse=np.sqrt(mean_squared_error(test,pred))
        
            order1.append(pdq)
            rmse.append(error_rmse)

        except:
            continue
            
    list(zip(rmse,mape))
    
    results=pd.DataFrame(index=order1,data=rmse,columns=['RMSE'])
    results.sort_values(by='RMSE',inplace=True)
    print("p,d,q values are",results.index[0]," with corresponding least RSME value of",results.RMSE[0])
    
    d=list(results.index[0])
    d.append(12)
    
    model_SAR=sm.tsa.statespace.SARIMAX(data ,order=results.index[0],seasonal_order=d)
    results=model_SAR.fit()
    
    pred=results.predict(start=len(train),end=(len(data)-1))
    
    pred_plot = results.predict(start=len(data)-1,end=(len(data)-1+prediction_num))
    
    data.plot(legend=True,label='Train',figsize=(15,8))

    pred_plot.plot(legend=True,label='Prediction',title="Prediction plot")
    
    


In [ ]:
SARIMA_MODEL("Uttarakhand",12)

In [ ]:
SARIMA_MODEL("Haryana",12)

In [ ]:
SARIMA_MODEL("Uttar Pradesh",12)

In [ ]:
SARIMA_MODEL("Himachal Pradesh",12)

In [ ]:
SARIMA_MODEL("Punjab",12)